In [1]:
import os

os.chdir('..')

In [2]:
from statsmodels.tsa.ar_model import AutoReg

from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score, recall_score, precision_score
import random
import csv
import pandas as pd
import numpy as np
import pickle
from math import sqrt

In [8]:
housing_index = "data/real_estate_data/direct/england.csv"
housing_column = "House price index"
#housing_index = "data/real_estate_data/securitized/FTSE EPRA_NAREIT UK Historical Data.csv"
#housing_column = "Price"

In [9]:
housing_data = pd.read_csv(housing_index, parse_dates=['Period'], index_col='Period',
                                        usecols=['Period', housing_column], thousands=',')

#housing_data.index = pd.to_datetime(housing_data.index, format="%b %y")

housing_data.index = housing_data.index.to_period("M")
housing_data = housing_data.sort_index()
housing_data[housing_column] = housing_data[housing_column].astype(float)
housing_data = housing_data.diff().dropna()

In [10]:
n_test = round(len(housing_data) * 0.2)
training_data, test_data = housing_data[0:-n_test], housing_data[-n_test:]

In [12]:
model = AutoReg(list(training_data[housing_column]), lags=2)
model_fit = model.fit()
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:                      y   No. Observations:                  192
Model:                     AutoReg(2)   Log Likelihood                -160.173
Method:               Conditional MLE   S.D. of innovations              0.562
Date:                Sun, 24 May 2020   AIC                             -1.110
Time:                        15:50:36   BIC                             -1.041
Sample:                             2   HQIC                            -1.082
                                  192                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.1048      0.046      2.266      0.023       0.014       0.195
y.L1           0.5675      0.072      7.921      0.000       0.427       0.708
y.L2           0.1482      0.072      2.069      0.039       0.008       0.289
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.3123           +0.0000j            1.3123            0.0000
AR.2           -5.1406           +0.0000j            5.1406            0.5000
-----------------------------------------------------------------------------
"""

In [9]:
"{:.3e}".format(0.289)

'2.890e-01'

In [46]:
predictions = model_fit.predict(start=len(training_data), end=len(training_data)+len(test_data)-1, dynamic=False)
actuals = list(test_data[housing_column])

In [47]:
mape = 0
for i in range(len(predictions)):
    actual = actuals[i]
    predicted = predictions[i]
    mape += abs((actual-predicted)/actual)
    print('predicted=%f, expected=%f' % (predicted, actual))

mape = mape / len(predictions)
rmse = sqrt(mean_squared_error(test_data, predictions))
print('Test RMSE: %.3f' % rmse)
print('Test MAPE: %.3f' % mape)

predicted=-0.586570, expected=-90.370000
predicted=0.663052, expected=-14.020000
predicted=2.923949, expected=99.800000
predicted=3.306773, expected=-77.500000
predicted=3.497813, expected=103.720000
predicted=3.547236, expected=-16.360000
predicted=3.565689, expected=77.410000
predicted=3.571229, expected=-9.950000
predicted=3.573115, expected=-45.950000
predicted=3.573711, expected=25.370000
predicted=3.573908, expected=-9.530000
predicted=3.573971, expected=-27.790000
predicted=3.573992, expected=-8.300000
predicted=3.573999, expected=-2.530000
predicted=3.574001, expected=139.010000
predicted=3.574002, expected=-57.130000
predicted=3.574002, expected=-92.290000
predicted=3.574002, expected=58.020000
predicted=3.574002, expected=82.050000
predicted=3.574002, expected=-14.670000
predicted=3.574002, expected=-0.180000
predicted=3.574002, expected=-14.090000
predicted=3.574002, expected=-43.290000
predicted=3.574002, expected=-48.990000
predicted=3.574002, expected=-33.490000
predicted

In [48]:
accuracy_score(np.sign(actuals), np.sign(predictions))

0.3953488372093023

In [49]:
confusion_matrix(np.sign(actuals), np.sign(predictions))

array([[ 1, 26],
       [ 0, 16]])

In [50]:
precision_score(np.sign(actuals), np.sign(predictions))

0.38095238095238093